In [ ]:
import generate

In [ ]:
import pandas as pd

df_one=pd.read_csv("one_step.csv")
df_two=pd.read_csv("two_step.csv")
for i in range(20):
    print(str(df_one["input"][i])+"\n")
    print("one step:"+str(df_one["response"][i]))
    print("two step:"+str(df_two["response"][i])+"\n\n\n")

In [1]:
prompt_input ="Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n"
print(prompt_input)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:



In [1]:
import openai
import os
import credentials
os.environ["OPENAI_API_KEY"] = credentials.api_key
openai.api_key = os.getenv("OPENAI_API_KEY")
from utils.prompter import Prompter
import pandas as pd
from tqdm import tqdm

cables=pd.read_csv("../amazon-dataset/cables.csv")
amps=pd.read_csv("../amazon-dataset/home_audio.csv")
teles=pd.read_csv("../amazon-dataset/televisions.csv")
cables = cables.sample(frac=1, random_state=10).reset_index(drop=True)[:20]
amps = amps.sample(frac=1, random_state=10).reset_index(drop=True)[:20]
teles = teles.sample(frac=1, random_state=10).reset_index(drop=True)[:20]

eval_prompt="""
[Instruction]
Please act as an impartial judge and evaluate the quality of the response provided by an AI assistant to the user question displayed below. Your evaluation should consider factors such as the helpfulness, relevance, accuracy, depth, creativity, and level of detail of the response. Begin your evaluation by providing a short explanation. Be as objective as possible. After providing your explanation, you must rate the response on a scale of 1 to 10 by strictly following this format: \"[[rating]]\", for example: \"Rating: [[5]]\".

[Question]
I'll give you as "input" a sequence of products with their functionalities. Each product is in the form:

<product> {product name}
<features> {list of features of the product}

You have to write a description of a luxury ship room containing these products. Do not copy the features, try to focus on the user experience instead of the technical details.
Write it with an engaging tone for the ship website.


"""

prod_list=[prod for prod in amps["title"]]+[prod for prod in teles["title"]]+[prod for prod in cables["title"]]
spec_list=[feat for feat in amps["feature"]]+[feat for feat in teles["feature"]]+[feat for feat in cables["feature"]]
input=list()
assert len(prod_list)==len(spec_list)
for i in tqdm(range(20)):
    input.append("<product> "+prod_list[i]+"\n"+"<features> "+spec_list[i]+"\n\n"+"<product> "+prod_list[20+i]+"\n"+"<features> "+spec_list[20+i]+"\n\n"+"<product> "+prod_list[40+i]+"\n"+"<functionalities> "+spec_list[40+i]+"\n\n")


100%|██████████| 20/20 [00:00<00:00, 191520.73it/s]


In [ ]:
one_step_response=pd.read_csv("./one_step.csv")["response"]
two_step_response=pd.read_csv("./two_step.csv")["response"]
one_val_list=list()
two_val_list=list()
for i in tqdm(range(20)):
    one_prompt=eval_prompt+"Input:\n"+input[i]+"[The Start of Assistant's Answer]\n"+one_step_response[i]+"\n[The End of Assistant's Answer]"
    one_val = openai.ChatCompletion.create(
        model="gpt-4",
        messages= [ {"role": "system","content": "You are a helpful assistant."},
                    {"role": "user","content": one_prompt}])
    one_val_list.append(int(one_val["choices"][0]["message"]["content"].split("[[")[1].split("]]")[0]))
    two_prompt=eval_prompt+"Input:\n"+input[i]+"[The Start of Assistant's Answer]\n"+two_step_response[i]+"\n[The End of Assistant's Answer]"
    two_val = openai.ChatCompletion.create(
        model="gpt-4",
        messages= [ {"role": "system","content": "You are a helpful assistant."},
                    {"role": "user","content": two_prompt}])
    two_val_list.append(int(two_val["choices"][0]["message"]["content"].split("[[")[1].split("]]")[0]))

In [53]:
one_df=pd.read_csv("./two_step.csv")
one_df["eval"]=one_val_list
one_df.to_csv("two_step.csv",index=False)

In [54]:
import numpy as np

df_one=pd.read_csv("./one_step.csv")
df_two=pd.read_csv("./two_step.csv")
print(np.mean(df_one["eval"]))
print(np.mean(df_two["eval"]))

In [10]:
prompt_gen="""generate a better prompt for my model. Output only the new generated prompt. Don't forget to describe the structure of the input

I'll give you as "input" a sequence of products with their functionalities. Each product is in the form:

<product> {product name}
<features> {list of features of the product}

You have to write a description of a luxury ship room containing these products. Do not copy the features, try to focus on the user experience instead of the technical details.
Write it with an engaging tone for the ship website.

"""

response = openai.ChatCompletion.create(
        model="gpt-4",
        messages= [ {"role": "system","content": "You are a helpful assistant."},
                    {"role": "user","content": prompt_gen}])

In [1]:
import pandas as pd

df=pd.read_csv("prompt_evaluations.csv",index_col=False)
#df=df.sort_values("mean_value")
df

,prompt,results,mean_value
0,"\nI'll give you as ""input"" a sequence of produ...","[7.0, 5.0, 8.0, 4.0, 4.0, 7.0, 3.0, 8.0, 4.0, ...",6.050
1,"""As an AI model, your task is to transform a s...","[2.0, 9.0, 3.0, 4.0, 4.5, 3.0, 9.0, 7.0, 1.0, ...",4.875
2,"""\nIn the 'input', you will receive a series o...","[8.0, 8.5, 6.0, 7.0, 8.5, 4.0, 9.0, 4.0, 4.0, ...",6.575
3,"""In the 'input', you will encounter a selectio...","[8.5, 1.0, 7.0, 8.0, 6.8, 8.5, 3.0, 7.0, 7.0, ...",6.165
4,"""In the 'input', you will receive a collection...","[9.8, 8.7, 6.5, 9.0, 2.0, 4.0, 8.0, 5.0, 4.0, ...",6.400
5,"""In the 'input', you are given a series of ups...","[6.0, 7.0, 9.0, 4.0, 8.0, 7.0, 6.5, 7.0, 8.0, ...",6.360
6,"""In the 'input', you will receive a diverse co...","[2.0, 10.0, 7.0, 5.0, 3.0, 4.0, 3.0, 6.0, 5.0,...",5.175
7,"""In the 'input', you are given a range of exqu...","[8.0, 5.0, 8.0, 7.0, 3.0, 8.0, 9.0, 7.0, 9.0, ...",6.250
8,"""In the 'input', you are given details about v...","[3.0, 6.0, 7.5, 8.0, 9.0, 8.0, 7.0, 6.0, 6.0, ...",6.600
9,"""In the input, you will receive detailed infor...","[3.0, 1.0, 4.0, 4.0, 3.0, 4.0, 5.0, 9.7, 5.0, ...",4.410
